In [1]:
import pandas as pd

In [2]:
data=pd.read_pickle("/content/2_CarPricesData.pkl")
data

,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,23.0,46986,1165.0,90,1,2000.0,3,13500
1,23.0,72937,1165.0,90,1,2000.0,3,13750
2,24.0,41711,1165.0,90,1,2000.0,3,13950
3,26.0,48000,1165.0,90,0,2000.0,3,14950
4,30.0,38500,1170.0,90,0,2000.0,3,13750
...,...,...,...,...,...,...,...,...
1431,69.0,20544,1025.0,86,1,1300.0,3,7500
1432,72.0,19000,1015.0,86,0,1300.0,3,10845
1433,71.0,17016,1015.0,86,0,1300.0,3,8500
1434,70.0,16916,1015.0,86,1,1300.0,3,7250


In [3]:
data.isnull().sum()

,0
Age,0
KM,0
Weight,0
HP,0
MetColor,0
CC,0
Doors,0
Price,0


In [4]:
data.duplicated().sum()

np.int64(0)

In [5]:
data.dtypes

,0
Age,float64
KM,int64
Weight,float64
HP,int64
MetColor,int64
CC,float64
Doors,int64
Price,int64


In [6]:
data.columns

Index(['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors', 'Price'], dtype='object')

In [7]:
data.shape

(1435, 8)

In [8]:
numeric_cols = ['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors', 'Price']

outlier_counts = {}

for col in numeric_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    outlier_counts[col] = len(outliers)

for col, count in outlier_counts.items():
    print(f"{col}: {count} outliers")

Age: 7 outliers
KM: 49 outliers
Weight: 65 outliers
HP: 11 outliers
MetColor: 0 outliers
CC: 121 outliers
Doors: 0 outliers
Price: 109 outliers


In [9]:

def remove_outliers_zscore(df, column, threshold=3):
    mean = data[column].mean()
    std = data[column].std()
    z_scores = (data[column] - mean) / std
    df_cleaned = data[(z_scores < threshold) & (z_scores > -threshold)].copy()
    return df_cleaned

for col in numeric_cols:
    data = remove_outliers_zscore(data, col)

data.shape

(1354, 8)

In [10]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
data[['Age','KM','Weight','HP','CC']] = sc.fit_transform(data[['Age','KM','Weight','HP','CC']])
data.head()

,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,0.278481,0.260107,0.804878,0.446809,1,1.0,3,13500
1,0.278481,0.403771,0.804878,0.446809,1,1.0,3,13750
2,0.291139,0.230905,0.804878,0.446809,1,1.0,3,13950
3,0.316456,0.265721,0.804878,0.446809,0,1.0,3,14950
4,0.367089,0.213129,0.829268,0.446809,0,1.0,3,13750


In [11]:
X=data.drop('Price',axis=1)
y=data['Price']

In [12]:
X.shape

(1354, 7)

In [13]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
X_train.shape


(1083, 7)

In [15]:
X_test.shape

(271, 7)

In [16]:
X_train.iloc[0]

,897
Age,0.822785
KM,0.332152
Weight,0.073171
HP,0.361702
MetColor,1.000000
CC,0.000000
Doors,3.000000


In [17]:
'''from keras import Sequential
from keras.layers import Dense
model=Sequential()
model.add(Dense(units=16,input_dim=7,kernel_initializer='random_uniform',activation='relu'))
model.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu',name='H1'))
model.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu',name='H2'))
model.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu',name='H3'))
model.add(Dense(units=16,kernel_initializer='he_uniform',activation='relu',name='H4'))
model.add(Dense(units=1,kernel_initializer='he_uniform'))'''



"from keras import Sequential\nfrom keras.layers import Dense\nmodel=Sequential()\nmodel.add(Dense(units=16,input_dim=7,kernel_initializer='random_uniform',activation='relu'))\nmodel.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu',name='H1'))\nmodel.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu',name='H2'))\nmodel.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu',name='H3'))\nmodel.add(Dense(units=16,kernel_initializer='he_uniform',activation='relu',name='H4'))\nmodel.add(Dense(units=1,kernel_initializer='he_uniform'))"

In [18]:
'''from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=5,restore_best_weights=True)'''



"from keras.callbacks import EarlyStopping, ModelCheckpoint\n\nearly_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=5,restore_best_weights=True)"

In [19]:
'''checkpoint=ModelCheckpoint(filepath='/content/Myfiles/Car_model_cp.keras',monitor='val_loss',verbose=1,save_best_only=True)'''

"checkpoint=ModelCheckpoint(filepath='/content/Myfiles/Car_model_cp.keras',monitor='val_loss',verbose=1,save_best_only=True)"

In [20]:
'''model.compile(loss='mse',optimizer='adam',metrics=['mse'])'''

"model.compile(loss='mse',optimizer='adam',metrics=['mse'])"

In [21]:
'''hist=model.fit(X_train,y_train,epochs=20,batch_size=40,verbose=1,validation_split=0.3,callbacks=[early_stop,checkpoint])'''

'hist=model.fit(X_train,y_train,epochs=20,batch_size=40,verbose=1,validation_split=0.3,callbacks=[early_stop,checkpoint])'

In [22]:
'''from keras import Sequential
from keras.layers import Dense
model2=Sequential()
model2.add(Dense(units=75,input_dim=7,kernel_initializer='random_uniform',activation='relu'))
model2.add(Dense(units=100,kernel_initializer='he_uniform',activation='relu',name='H1'))
model2.add(Dense(units=50,kernel_initializer='he_uniform',activation='relu',name='H2'))
model2.add(Dense(units=25,kernel_initializer='he_uniform',activation='relu',name='H3'))
model2.add(Dense(units=15,kernel_initializer='he_uniform',activation='relu',name='H4'))
model2.add(Dense(units=18,kernel_initializer='he_uniform',activation='relu',name='H5'))
model2.add(Dense(units=1,kernel_initializer='he_uniform'))'''



"from keras import Sequential\nfrom keras.layers import Dense\nmodel2=Sequential()\nmodel2.add(Dense(units=75,input_dim=7,kernel_initializer='random_uniform',activation='relu'))\nmodel2.add(Dense(units=100,kernel_initializer='he_uniform',activation='relu',name='H1'))\nmodel2.add(Dense(units=50,kernel_initializer='he_uniform',activation='relu',name='H2'))\nmodel2.add(Dense(units=25,kernel_initializer='he_uniform',activation='relu',name='H3'))\nmodel2.add(Dense(units=15,kernel_initializer='he_uniform',activation='relu',name='H4'))\nmodel2.add(Dense(units=18,kernel_initializer='he_uniform',activation='relu',name='H5'))\nmodel2.add(Dense(units=1,kernel_initializer='he_uniform'))"

In [23]:
'''from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=7,restore_best_weights=True)'''



"from keras.callbacks import EarlyStopping, ModelCheckpoint\n\nearly_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=7,restore_best_weights=True)"

In [24]:
'''checkpoint=ModelCheckpoint(filepath='/content/Myfiles/Car_model_cp2.keras',monitor='val_loss',verbose=1,save_best_only=True)'''

"checkpoint=ModelCheckpoint(filepath='/content/Myfiles/Car_model_cp2.keras',monitor='val_loss',verbose=1,save_best_only=True)"

In [25]:
'''model2.compile(loss='mse',optimizer='adam',metrics=['mse'])'''

"model2.compile(loss='mse',optimizer='adam',metrics=['mse'])"

In [26]:
'''history=model2.fit(X_train,y_train,epochs=100,batch_size=40,verbose=1,validation_split=0.2,callbacks=[early_stop,checkpoint])'''

'history=model2.fit(X_train,y_train,epochs=100,batch_size=40,verbose=1,validation_split=0.2,callbacks=[early_stop,checkpoint])'

In [27]:
'''model2.summary()'''

'model2.summary()'

In [28]:
'''from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

model3 = Sequential()
model3.add(Dense(units=75, input_dim=7, kernel_initializer='random_uniform', activation='relu'))
model3.add(Dense(units=90, kernel_initializer='he_uniform', activation='relu', name='H1'))
model3.add(Dense(units=100, kernel_initializer='he_uniform', activation='relu', name='H2'))
model3.add(Dense(units=85, kernel_initializer='he_uniform', activation='relu', name='H3'))
model3.add(Dense(units=70, kernel_initializer='he_uniform', activation='relu', name='H4'))
model3.add(Dense(units=50, kernel_initializer='he_uniform', activation='relu', name='H5'))
model3.add(Dense(units=30,kernel_initializer='he_uniform',activation='relu',name='H6'))
model3.add(Dense(units=20,kernel_initializer='he_uniform',activation='relu',name='H7'))
model3.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu',name='H8'))
model3.add(Dense(units=1, kernel_initializer='he_uniform'))'''





"from keras import Sequential\nfrom keras.layers import Dense\nfrom keras.callbacks import EarlyStopping, ModelCheckpoint\n\nmodel3 = Sequential()\nmodel3.add(Dense(units=75, input_dim=7, kernel_initializer='random_uniform', activation='relu'))\nmodel3.add(Dense(units=90, kernel_initializer='he_uniform', activation='relu', name='H1'))\nmodel3.add(Dense(units=100, kernel_initializer='he_uniform', activation='relu', name='H2'))\nmodel3.add(Dense(units=85, kernel_initializer='he_uniform', activation='relu', name='H3'))\nmodel3.add(Dense(units=70, kernel_initializer='he_uniform', activation='relu', name='H4'))\nmodel3.add(Dense(units=50, kernel_initializer='he_uniform', activation='relu', name='H5'))\nmodel3.add(Dense(units=30,kernel_initializer='he_uniform',activation='relu',name='H6'))\nmodel3.add(Dense(units=20,kernel_initializer='he_uniform',activation='relu',name='H7'))\nmodel3.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu',name='H8'))\nmodel3.add(Dense(units=1, 

In [29]:
'''early_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=8,restore_best_weights=True)'''

"early_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=8,restore_best_weights=True)"

In [30]:
'''checkpoint=ModelCheckpoint(filepath='/content/Myfiles/Car_model_cp3.keras',monitor='val_loss',verbose=1,save_best_only=True)'''

"checkpoint=ModelCheckpoint(filepath='/content/Myfiles/Car_model_cp3.keras',monitor='val_loss',verbose=1,save_best_only=True)"

In [31]:
''' model3.compile(loss='mae',optimizer='adam',metrics=['mae'])''

SyntaxError: incomplete input (<ipython-input-31-1920342223>, line 1)

In [ ]:
'''hits3=model3.fit(X_train,y_train,epochs=100,batch_size=40,verbose=1,validation_split=0.2,callbacks=[early_stop,checkpoint])'''

In [ ]:
'''model3.summary()''

In [32]:
from keras import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

model4 = Sequential()
model4.add(Dense(units=75, input_dim=7, kernel_initializer='random_uniform', activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(units=100, kernel_initializer='he_uniform', activation='relu', name='H2'))
model4.add(Dense(units=85, kernel_initializer='he_uniform', activation='relu', name='H3'))
model4.add(Dropout(0.2))
model4.add(Dense(units=50, kernel_initializer='he_uniform', activation='relu', name='H5'))
model4.add(Dense(units=30,kernel_initializer='he_uniform',activation='relu',name='H6'))
model4.add(Dropout(0.3))
model4.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu',name='H8'))
model4.add(Dense(units=1, kernel_initializer='he_uniform'))





/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
early_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=8,restore_best_weights=True)

In [34]:
checkpoint=ModelCheckpoint(filepath='/content/Myfiles/Car_model_cp4.keras',monitor='val_loss',verbose=1,save_best_only=True)

In [35]:
model4.compile(loss='mae',optimizer='adam',metrics=['mae'])

In [36]:
hist4=model4.fit(X_train,y_train,epochs=100,batch_size=40,verbose=1,validation_split=0.2,callbacks=[early_stop,checkpoint])

Epoch 1/100
14/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10558.8838 - mae: 10558.8838
Epoch 1: val_loss improved from inf to 10183.73242, saving model to /content/Myfiles/Car_model_cp4.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 10479.4033 - mae: 10479.4033 - val_loss: 10183.7324 - val_mae: 10183.7324
Epoch 2/100
15/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10393.5645 - mae: 10393.5645
Epoch 2: val_loss improved from 10183.73242 to 10037.60645, saving model to /content/Myfiles/Car_model_cp4.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 10374.2285 - mae: 10374.2285 - val_loss: 10037.6064 - val_mae: 10037.6064
Epoch 3/100
14/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10112.7959 - mae: 10112.7959 
Epoch 3: val_loss improved from 10037.60645 to 8127.22168, saving model to /content/Myfiles/Car_model_cp4.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 9980.0498 - mae: 9980.0498 - val_loss: 8127.2217 - val_mae: 8127.2217
Epoch 4/100
17/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [37]:
model4.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 75)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ H2 (Dense)                      │ (None, 100)            │         7,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ H3 (Dense)                      │ (None, 85)             │         8,585 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 85)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ H5 (Dense)                      │ (None, 50)             │         4,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ H6 (Dense)                      │ (None, 30)             │         1,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ H8 (Dense)                      │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,810 (268.79 KB)

 Trainable params: 22,936 (89.59 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 45,874 (179.20 KB)